In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, mean_squared_error
import xgboost as xgb

In [ ]:
#  Load dataset
df = pd.read_csv(r"C:\Users\ROG\OneDrive\Documents\DeskDump\Python\Parkinsons\pd_speech_features.csv")


# Drop ID column and separate features/target
X = df.drop(columns=['id', 'class'])
y = df['class']

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train-test split with stratification
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42, stratify=y
)

# Train XGBoost model
model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
model.fit(X_train, y_train)

# Predictions and evaluation
y_pred = model.predict(X_test)
print("Classification Report:\n", classification_report(y_test, y_pred))

print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))


c:\Users\ROG\AppData\Local\Programs\Python\Python312\Lib\site-packages\xgboost\core.py:158: UserWarning: [11:19:28] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Classification Report:
               precision    recall  f1-score   support

           0       0.78      0.64      0.70        39
           1       0.88      0.94      0.91       113

    accuracy                           0.86       152
   macro avg       0.83      0.79      0.81       152
weighted avg       0.86      0.86      0.86       152


Confusion Matrix:
[[ 25  14]
 [  7 106]]


In [4]:
mse_xgb = mean_squared_error(y_test,y_pred)
print(mse_xgb)

0.13815789473684212


In [5]:
y_train.value_counts()


class
1    451
0    153
Name: count, dtype: int64

In [6]:
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)
# Check resampled class distribution
print("\nResampled class distribution:")
print(pd.Series(y_train_resampled).value_counts())


Resampled class distribution:
class
1    451
0    451
Name: count, dtype: int64


In [7]:
model.fit(X_train_resampled, y_train_resampled)

# Predictions and evaluation
y_pred = model.predict(X_test)
print("Classification Report:\n", classification_report(y_test, y_pred))

print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))


c:\Users\ROG\AppData\Local\Programs\Python\Python312\Lib\site-packages\xgboost\core.py:158: UserWarning: [11:19:31] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Classification Report:
               precision    recall  f1-score   support

           0       0.76      0.67      0.71        39
           1       0.89      0.93      0.91       113

    accuracy                           0.86       152
   macro avg       0.83      0.80      0.81       152
weighted avg       0.86      0.86      0.86       152


Confusion Matrix:
[[ 26  13]
 [  8 105]]


In [ ]:
# from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor
# param_grid_xgb = {
#     'lambda': [0.01, 0.1, 1, 10],
#     'gamma': [0, 0.1, 1, 10],
#     'max_depth': [3, 5, 7],
#     'learning_rate': [0.01, 0.1, 0.2],
#     'n_estimators': [100, 200, 300]
# }

# # GridSearchCV for XGBRegressor (regression task => no accuracy score!)
# grid_search = GridSearchCV(
#     estimator=xgb.XGBRegressor(objective='reg:squarederror', random_state=42),
#     param_grid=param_grid_xgb,
#     scoring='neg_mean_squared_error',  # Use appropriate regression metric
#     cv=10,
#     n_jobs=-1
# )

from sklearn.model_selection import GridSearchCV

param_grid_xgb = {
    'lambda': [0.01, 0.1, 1, 10],
    'gamma': [0, 0.1, 1, 10],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
    'n_estimators': [100, 200, 300]
}

xgb_reg = xgb.XGBRegressor(
    objective='reg:squarederror',
    tree_method='gpu_hist',        # Enable GPU acceleration
    predictor='gpu_predictor',     # Optional: speeds up prediction too
    random_state=42
)

grid_search = GridSearchCV(
    estimator=xgb_reg,
    param_grid=param_grid_xgb,
    scoring='neg_mean_squared_error',
    cv=10,
    n_jobs=-1,
    verbose=2                        # Optional: shows progress
)


In [9]:
# Fit on training data (use x_train, y_train instead of full x and y)
grid_result = grid_search.fit(X_train, y_train)

# Display the best parameters and the (negative) MSE score
print(f"Best MSE (negative): {grid_result.best_score_}")
print(f"Best parameters: {grid_result.best_params_}")

Fitting 5 folds for each of 288 candidates, totalling 1440 fits


KeyboardInterrupt: 